<div style="text-align: right" align="right"><i>Ivy Zhang, December 2023</i></div>

# Advent of Code 2023

Going to try and solve Advent of Code as efficiently as possible this year to get better at Python!

## Day 0: Prep

Just importing things that I might need to keep this file clean!

In [1]:
%run Advent-Utils.ipynb

## [Day 1](https://adventofcode.com/2023/day/1): Trebuchet?!

### Part 1: Find first and last digit, concatenate and sum all of them together

In [2]:
in1 = parse(1)

numbers1 = [re.findall(r'\d', t) for t in in1]
combined_numbers1 = [int(str(t[0]) + str(t[-1])) for t in numbers1]
ans1 = sum(combined_numbers1)

print(ans1)

54968


### Part 2: You must now also take text digits into consideration

In [3]:
mappings = {
    'one': '1',
    'two': '2',
    'three': '3',
    'four': '4',
    'five': '5',
    'six': '6',
    'seven': '7',
    'eight': '8',
    'nine': '9',
}

def parse_calibration(cal, iter):
    for i in iter:
        if cal[i].isdigit():
            return int(cal[i])
        for dig in mappings:
            if cal[i:i+len(dig)] == dig:
                return mappings[dig]


def parse_calibrations(calibrations):
    combined_nums = []
    for cal in calibrations:
        fd, ld = -1, -1
        fd = parse_calibration(cal, range(len(cal)))
        ld = parse_calibration(cal, reversed(range(len(cal))))
        combined_nums.append([fd, ld])
    return combined_nums

in1 = parse(1)
in1 = parse_calibrations(in1)
combined_numbers1 = [int(str(t[0]) + str(t[-1])) for t in in1]
ans1 = sum(combined_numbers1)
print(ans1)


54094


## [Day 2](https://adventofcode.com/2023/day/2): Cube Conundrum

In [4]:
in2 = parse(2, atoms)

### Part 1

Each "hand" can be played in any order and with number of colors. This means my parsing has to be able to take that into account. Given 12 red cubes, 13 blue cubes, and 14 green cubes, see is the hands that the elf is showing are valid

In [5]:
limits = {
    'red': 12,
    'green': 13,
    'blue': 14
}

def find_cube_possibilities() -> int:
    ans = 0
    for game in in2:
        id = game[1]
        for i in range(2, len(game), 2):
            num, color = game[i], game[i+1]
            if num > limits[color]:
                break
        else:
            ans += id
    return ans

print(find_cube_possibilities())

2600


### Part 2

Instead of checking whether or not a game is possible, figure out what the minimum number of each type of color is.

In [6]:
def find_min_cubes() -> int:
    ans = 0
    for game in in2:
        color_mins = {
            'red': 0,
            'green': 0,
            'blue': 0
        }
        for i in range(2, len(game), 2):
            num, color = game[i], game[i+1]
            color_mins[color] = max(color_mins[color], num)
            
        game_power = 1
        for color in color_mins:
            game_power *= color_mins[color]
        ans += game_power
    return ans

print(find_min_cubes())

86036


### [Day 3](https://adventofcode.com/2023/day/3): Gear Ratios

In [7]:
in3 = list(parse(3))

One idea is that from the symbols, you flood fill outwards going into number tiles, then you can use regex to find the numbers that are "valid".

In [8]:
h, l = len(in3), len(in3[0])

val = [[0 for _ in range(l)] for _ in range(h)]
q = deque()

for i, line in enumerate(in3):
    for j, c in enumerate(line):
        if not c.isdigit() and c != '.':
            for di in range(-1, 2):
                for dj in range(-1, 2):
                    ni, nj = i + di, j + dj
                    if 0 <= ni < h and 0 <= nj < l and in3[ni][nj].isdigit():
                        val[ni][nj] = 1
                        q.append((ni, nj))

while len(q) > 0:
    i, j = q.pop()
    # print('popping')
    for dj in range(-1, 2):
        ni, nj = i, j + dj
        if 0 <= ni < h and 0 <= nj < l:
            # print(f'({ni}, {nj}) -> {in3[ni][nj]}')
            if val[ni][nj] == 0 and in3[ni][nj].isdigit():
                # print('adding new')
                val[ni][nj] = 1
                # print(f'Adding ({ni}, {nj})')
                q.append((ni, nj))

ans = 0

# for i in range(h):
#     for j in range(l):
#         print(val[i][j], end='')
#     print()

for i, line in enumerate(in3):
    nums = [(m.group(0), m.start(0)) for m in re.finditer(r'\d+', line)]
    for num, index in nums:
        if val[i][index] > 0:
            ans += int(num)
            # print(f'Adding {num}')

print(ans)


550934


### Part 2

Now we're looking for gears. Instead of flooding each one, should write a function that takes in an index and a marker and floods it in along with giving you the integer. Then for each symbol, keep track of how many ints you got to see whether or not you have a valid "gear".

In [9]:
h, l = len(in3), len(in3[0])

vis = [[0 for _ in range(l)] for _ in range(h)]

def flood_num(i, j):
    val = in3[i][j]
    lj, rj = j-1, j+1
    vis[i][j] = 1

    while lj >= 0 and in3[i][lj].isdigit():
        val = in3[i][lj] + val
        vis[i][lj] = 1
        lj -= 1

    while rj < l and in3[i][rj].isdigit():
        val = val + in3[i][rj]
        vis[i][rj] = 1
        rj += 1

    return int(val)

ans = 0

for i, line in enumerate(in3):
    for j, c in enumerate(line):
        if c == '*':
            adjacent_nums = []
            for di in range(-1, 2):
                for dj in range(-1, 2):
                    ni, nj = i + di, j + dj
                    if 0 <= ni < h and 0 <= nj < l and in3[ni][nj].isdigit() and not vis[ni][nj]:
                        adjacent_nums.append(flood_num(ni, nj))
            if len(adjacent_nums) == 2:
                ans += adjacent_nums[0] * adjacent_nums[1]

print(ans)


81997870


## [Day 4](https://adventofcode.com/2023/day/4): Scratchcards

In [10]:
in4 = parse(4, pos_ints)

In [11]:
print(in4)

((1, 33, 56, 23, 64, 92, 86, 94, 7, 59, 13, 86, 92, 64, 43, 10, 70, 16, 55, 79, 33, 56, 8, 7, 25, 82, 14, 31, 96, 94, 13, 99, 29, 69, 75, 23), (2, 61, 66, 75, 1, 27, 38, 93, 90, 34, 43, 94, 46, 62, 49, 35, 88, 45, 70, 15, 22, 20, 86, 56, 38, 64, 98, 50, 6, 79, 11, 13, 93, 92, 60, 16), (3, 57, 7, 33, 56, 85, 6, 88, 34, 80, 8, 92, 42, 7, 60, 61, 51, 40, 6, 67, 35, 3, 25, 87, 2, 98, 75, 97, 54, 10, 68, 73, 83, 4, 62, 56), (4, 79, 85, 94, 74, 15, 62, 84, 88, 76, 56, 56, 9, 22, 57, 4, 92, 62, 79, 84, 64, 72, 55, 34, 88, 66, 15, 45, 18, 76, 73, 85, 94, 8, 78, 74), (5, 57, 94, 99, 25, 52, 67, 69, 31, 26, 78, 94, 52, 31, 83, 70, 45, 40, 67, 89, 11, 81, 24, 25, 61, 26, 72, 50, 12, 27, 69, 91, 57, 55, 34, 78), (6, 5, 96, 3, 19, 98, 25, 13, 59, 94, 8, 36, 55, 22, 76, 86, 19, 10, 8, 59, 9, 87, 40, 2, 71, 13, 98, 12, 77, 3, 70, 5, 25, 34, 41, 88), (7, 35, 52, 84, 36, 72, 53, 76, 88, 41, 14, 57, 34, 14, 39, 44, 71, 51, 1, 67, 30, 16, 77, 23, 66, 45, 74, 37, 55, 38, 69, 33, 31, 98, 72, 36), (8, 7, 70

In [12]:
ans = 0
num_winning = 10

for game in in4:
    winners = set(game[1:1+num_winning])
    matches = sum([int(x in winners) for x in game[num_winning+1:]])
    prize = matches if matches <= 1 else int(pow(2, matches-1))
    ans += prize

print(ans)

23441


### Part 2

Rather than points, the cards multiply next ones depending on how many matches they got

In [13]:
ans = 0
num_winning = 10
copies = 1

in4 = list(in4)
pre_sums = [0 for _ in range(len(in4))]

for i, game in enumerate(in4):
    copies += pre_sums[i]
    winners = set(game[1:1+num_winning])
    matches = sum([int(x in winners) for x in game[num_winning+1:]])

    li, ri = i+1, i+matches+1
    if li < len(pre_sums):
        pre_sums[li] += copies
    if ri < len(pre_sums):
        pre_sums[ri] -= copies
    ans += copies

print(ans)

5923918


## [Day 5](https://adventofcode.com/2023/day/5): If you give a seed a fertilizer

In [14]:
in5 = parse(5, pos_ints, paragraphs)

In [15]:
seeds = list(in5[0])

for i in range(1, len(in5)):
    mappings = []
    for j in range(0, len(in5[i]), 3):
        mappings.append((in5[i][j], in5[i][j+1], in5[i][j+2]))
    for i in range(len(seeds)):
        for dst, src, rge in mappings:
            if src <= seeds[i] < src + rge:
                seeds[i] = dst + (seeds[i] - src)
                break

print(min(seeds))

551761867


#### Part 2

So now instead of having small numbers of seeds, we need to consider large swaths of seeds, you can just individually process them anymore. You can consider each transformation as shifting a range of source values by some amount, and this is linear. Now the ranges can be MASSIVE, so you can't just do any iteration over the # of seeds.

One easy way to do this is just to maintain ranges then? If you do that, then you can modify what everything maps to? This hopefully won't grow out of control

In [16]:
seeds = list(in5[0])
seed_ranges = sorted([(seeds[i], seeds[i]+seeds[i+1]-1) for i in range(0, len(seeds), 2)])

# print('intial range', seed_ranges)
for i in range(1, len(in5)):
    mappings = []
    for j in range(0, len(in5[i]), 3):
        # how much to shift range, src start, src range,
        mappings.append((in5[i][j]-in5[i][j+1], in5[i][j+1], in5[i][j+2]))

    # print(mappings)
    new_seed_ranges = []
    # print(seed_ranges)
    pre_sums = defaultdict(int)
    for j in range(len(seed_ranges)):
        # print(j, seed_ranges[j])
        lb, rb = seed_ranges[j]
        pre_sums[lb] = 0; pre_sums[rb+1] = 0
        for diff, src, rge in mappings:
            # Check to see if mapping range intersects at all
            if lb <= src <= rb or src <= lb < src+rge:
                pre_sums[max(src, lb)] += diff
                pre_sums[min(rb+1, src+rge)] += -diff

        # convert single range to more ranges
        range_changes = sorted(pre_sums.items())
        # print('range changes', range_changes)
        rge_start, accum_diff = range_changes[0]
        for k in range(1, len(range_changes)):
            new_rge_start, new_diff = range_changes[k]            
            new_seed_ranges.append((rge_start+accum_diff, new_rge_start-1+accum_diff))
            rge_start = new_rge_start
            accum_diff += new_diff
        pre_sums.clear()
    
    new_seed_ranges = sorted(new_seed_ranges)
    # Merge ranges that are next to each other
    for j in range(len(new_seed_ranges)-1, 0, -1):
        # if this range starts one after next range ends, merge them
        if new_seed_ranges[j][0] == new_seed_ranges[j-1][1] + 1:
            new_seed_ranges[j-1] = (new_seed_ranges[j-1][0], new_seed_ranges[j][1])
            del new_seed_ranges[j]
    seed_ranges = new_seed_ranges


print(min(seed_ranges)[0])

57451709


## [Day 6](https://adventofcode.com/2023/day/6): Wait For It 

In [17]:
in6 = parse(6, pos_ints)

In [18]:
def f(hold_t: int, tot_t: int) -> int:
    return (tot_t - hold_t) * hold_t

In [19]:
def b_search(lo: int, hi: int, d: int) -> int:
    tot_time = hi
    while lo < hi:
        # Converge to one value
        if lo + 1 == hi:
            if f(lo, tot_time) > d:
                hi = lo
            else:
                lo = hi
            break

        mid = (lo + hi)//2
        if f(mid, tot_time) > d:
            hi = mid
        else:
            lo = mid + 1
    # Subtract one since you can never hold for 0 seconds (I think)
    return ((tot_time + 1) - (2 * lo))


In [20]:
times, distances = in6
options = [b_search(0, times[i], distances[i]) for i in range(len(times))]
print(math.prod(options))


1413720


#### Part 2

Instead of individual races, it's now just one big race. I should instead take in all the numbers, make them strings and them concatenate them all together.

In [21]:
times, distances = in6

# Convert to one big integer now
one_time = int(reduce(lambda x, y: x+y, mapt(str, times)))
one_dist = int(reduce(lambda x, y: x+y, mapt(str, distances)))
print(b_search(0, one_time, one_dist))

30565288


## [Day 7](https://adventofcode.com/2023/day/7): Camel Cards

In [22]:
in7 = parse(7, strings)

In [23]:
card_vals = "2 3 4 5 6 7 8 9 T J Q K A".split()

class hand(tuple):
    def __lt__(self, other):
        self_cards = Counter(self[0])
        other_cards = Counter(other[0])

        # If this hand has more unique cards, then this is the worse hand
        if len(self_cards) != len(other_cards):
            return len(self_cards) > len(other_cards)

        self_mc = self_cards.most_common()
        other_mc = other_cards.most_common()

        self_mc = sorted(self_mc, key=lambda x: pow(20, x[1]) + card_vals.index(x[0]), reverse=True)
        other_mc = sorted(other_mc, key=lambda x: pow(20, x[1]) + card_vals.index(x[0]), reverse=True)

        # But with this is that counter doesn't sort by "card" value
        for i in range(len(self_mc)):
            # If your current set of cards is not equal, then this hand is worse is you have less
            if self_mc[i][1] != other_mc[i][1]:
                return self_mc[i][1] < other_mc[i][1]

        for i in range(len(self[0])):
            if self[0][i] != other[0][i]:
                return card_vals.index(self[0][i]) < card_vals.index(other[0][i])

def hand_val(hand: tuple) -> int:
    cards = Counter(hand[0])


In [24]:
print(in7)
sorted_hands = sorted(in7, key=hand)
hand_vals = [int(sorted_hands[i][1]) * (i+1) for i in range(len(sorted_hands))]
print(sum(hand_vals))

(('K99QT', '53'), ('TKQ7T', '320'), ('22A7J', '490'), ('267J9', '69'), ('665JJ', '431'), ('K856J', '605'), ('977K9', '552'), ('KKK8K', '285'), ('53697', '370'), ('3J337', '879'), ('84A9K', '901'), ('K4289', '211'), ('4JA46', '100'), ('K452K', '315'), ('3K48Q', '239'), ('99TT9', '985'), ('A7JA9', '582'), ('836AT', '770'), ('J2KKK', '61'), ('KK7KK', '37'), ('QTAJK', '351'), ('T4578', '198'), ('TK55T', '513'), ('44K9J', '853'), ('KQT58', '203'), ('66A6J', '881'), ('K77KK', '31'), ('6666Q', '410'), ('8J8KJ', '522'), ('69KQ7', '594'), ('T27J3', '110'), ('44999', '461'), ('98885', '231'), ('55655', '372'), ('43343', '958'), ('82247', '300'), ('33336', '957'), ('59JT4', '303'), ('AQAJA', '691'), ('J3626', '696'), ('9379A', '788'), ('JTTTA', '795'), ('442A6', '989'), ('73933', '857'), ('979QQ', '817'), ('65566', '150'), ('6T565', '182'), ('4T5J5', '9'), ('TT3TT', '147'), ('J8338', '229'), ('6AQ8Q', '636'), ('79KK7', '933'), ('8T6TT', '145'), ('TTT4T', '177'), ('KAK52', '952'), ('9K83J', '469')

### Part 2

Jokers can act as anything, but now they're weak. Just when you check, add jokers to the top set, and now also move them below 2s in terms of individual value.

In [25]:
card_vals = "J 2 3 4 5 6 7 8 9 T Q K A".split()

class hand(tuple):
    def __lt__(self, other):
        self_cards = Counter(self[0])
        other_cards = Counter(other[0])

        self_mc = self_cards.most_common()
        other_mc = other_cards.most_common()

        self_js = self_cards['J']
        other_js = other_cards['J']

        self_mc = sorted(self_mc, key=lambda x: pow(20, x[1]) + card_vals.index(x[0]), reverse=True)
        other_mc = sorted(other_mc, key=lambda x: pow(20, x[1]) + card_vals.index(x[0]), reverse=True)

        if 0 < self_js < 5:
            self_mc.remove(('J', self_js))
            self_mc[0] = (self_mc[0][0], self_mc[0][1] + self_js)
        if 0 < other_js < 5:
            other_mc.remove(('J', other_js))
            other_mc[0] = (other_mc[0][0], other_mc[0][1] + other_js)

        if len(self_mc) != len(self_mc):
            return len(self_mc) > len(self_mc)

        # But with this is that counter doesn't sort by "card" value
        for i in range(len(self_mc)):
            # If your current set of cards is not equal, then this hand is worse is you have less
            if self_mc[i][1] != other_mc[i][1]:
                return self_mc[i][1] < other_mc[i][1]

        for i in range(len(self[0])):
            if self[0][i] != other[0][i]:
                return card_vals.index(self[0][i]) < card_vals.index(other[0][i])

def hand_val(hand: tuple) -> int:
    cards = Counter(hand[0])


In [26]:
sorted_hands = sorted(in7, key=hand)
hand_vals = [int(sorted_hands[i][1]) * (i+1) for i in range(len(sorted_hands))]
print(sum(hand_vals))

248750699


## [Day 8](https://adventofcode.com/2023/day/8): Haunted Wasteland

In [27]:
in8 = parse(8, strings)

In [28]:
ans = 0
cur_node = 'AAA'

steps, _, *connections = in8
steps = steps[0]

links = {}
for connection in connections:
    # print(connection)
    root, left, right = connection
    links[root] = (left, right)

while cur_node != 'ZZZ':
    # print(cur_node, ans % len(steps), steps[ans % len(steps)])
    cur_node = links[cur_node][steps[ans % len(steps)] == 'R']
    # print(c
    ans += 1

print(ans)

22411


### Part 2

Now working on Part 2, kind of confused about how to make it run quick enough. So you have to check when the first time is that all the nodes will land on a node that ends in Z. Maybe something you can do is you can find all the timesteps that a node will land on a Z thing, then you know you can add len(steps) to that for the next time it will be there. This would be quicker than stepping through all of them manually. Then you can check for the most common one, see if it is the all number of nodes, then continue on? Oh wait, this is incorrect, because then you'd have to know when a "cycle" of a node ends, which I could find but could also be quite long?

I sort of want to find GCD, but I don't think I really want to get the minimum steps and then repeat that? Also you can't really get GCD here. So for each node, I can find "long" a loop is, then I can get the number of times they stepped on a node with Z. Then I iterate through this? I suppose it is quicker than what I'm doing right now.

In [29]:

# num_steps = len(steps)



# def advance_node(node: str, step: int):
#     return links[node][steps[step % num_steps] == 'R']

# def advance_nodes(nodes: list, step: int) -> bool:
#     nodes = list(map(lambda n: advance_node(n, step), nodes))
#     print(nodes)
#     done = all([nodes[i][-1] == 'Z' for i in range(len(nodes))])
#     return nodes, done

# def find_cycles(node: str):
#     cur_step = 0
#     z_steps = []
#     visited = defaultdict(list)
    
#     while True:
#         # print(node)
#         node = advance_node(node, cur_step)
#         cur_step += 1
#         if (cur_step % num_steps) in visited[node]:
#             break
#         else:
#             if node[-1] == 'Z':
#                 z_steps.append(cur_step)
#             visited[node].append(cur_step % num_steps)
    
#     # Cycle length, when we get to it :)
#     return (cur_step - 1, z_steps)

# ans = 0
# nodes = [x for x in links if x[-1] == 'A']
# num_nodes = len(nodes)
# cycles = []
# done = False

# for node in nodes:
#     cycle = find_cycles(node)
#     cycles.append(cycle)
#     print(cycle)

# val = Counter()
# i = 0
# MAX_VALUE = 1_000_000_000_000
# min_ans = MAX_VALUE

# while not done:
#     for cycle in cycles:
#         cycle_len, z_steps = cycle
#         z_steps = [x+cycle_len * i for x in z_steps]
#         val = val + Counter(z_steps)
#     mc = val.most_common()
#     for z_step, z_cnt in mc:
#         if z_cnt != num_nodes:
#             break
#         else:
#             min_ans = min(min_ans, z_step)

#     if min_ans != MAX_VALUE:
#         print(min_ans)
#         break
    
#     i += 1


# # print(ans)



## [Day 9](https://adventofcode.com/2023/day/9): Mirage Maintenance

In [30]:
in9 = parse(9, atoms)

In [31]:
def next_value(history: list) -> int:
    done_iter = all(x == 0 for x in history)
    if not done_iter:
        next_history = [(history[i] - history[i-1]) for i in range(1, len(history))]
        return history[-1] + next_value(next_history)
    else:
        return 0

In [32]:
print(sum([next_value(x) for x in in9]))


1930746032


### Part 2

Now they want you to extrapolate backwards which should be quite easy with this recursion

In [33]:
print(sum([next_value(x[::-1]) for x in in9]))

1154


## [Day 10](https://adventofcode.com/2023/day/10): Pipe Maze

In [34]:
in10 = parse(10, chars)

In [35]:
def find_source(grid, letter):
    for i in range(len(grid)):
        for j in range(len(grid[i])):
            if grid[i][j] == letter:
                return (i, j)

In [36]:
source = find_source(in10, 'S')

ans = 0
visited = [[-1 for _ in range(len(in10[i]))] for i in range(len(in10))]

# At source, find all adjacent that can link up
visited[source[0]][source[1]] = 0


q = deque()
for (ni, nj) in cells_around_4(source[0], source[1], False):
    if ni < 0 or nj < 0 or ni >= len(in10) or nj >= len(in10[ni]):
        continue
    di, dj = ni - source[0], nj - source[1]
    if dj == 1 and in10[ni][nj] in ('7', 'J', '-'):
        visited[ni][nj] = 1
        q.append((ni, nj))
    elif dj == -1 and in10[ni][nj] in ('F', 'L', '-'):
        q.append((ni, nj))
        visited[ni][nj] = 1
    elif di == -1 and in10[ni][nj] in ('7', 'F', '|'):
        q.append((ni, nj))
        visited[ni][nj] = 1
    elif di == 1 and in10[ni][nj] in ('J', 'L', '|'):
        q.append((ni, nj))
        visited[ni][nj] = 1

In [37]:

while len(q):
    (i, j) = q.popleft()
    c = in10[i][j]

    # print(i, j)
    if c == '|':
        for di, dj in ((1, 0), (-1, 0)):
            ni, nj = i + di, j + dj
            if ni < 0 or nj < 0 or ni >= len(in10) or nj >= len(in10[ni]):
                continue
            if visited[ni][nj] > -1:
                continue
            q.append((ni, nj))
            visited[ni][nj] = visited[i][j] + 1
    elif c == '-':
        for di, dj in ((0, 1), (0, -1)):
            ni, nj = i + di, j + dj
            if ni < 0 or nj < 0 or ni >= len(in10) or nj >= len(in10[ni]):
                continue
            if visited[ni][nj] > -1:
                continue
            q.append((ni, nj))
            visited[ni][nj] = visited[i][j] + 1
    elif c == 'L':
        for di, dj in ((-1, 0), (0, 1)):
            ni, nj = i + di, j + dj
            if ni < 0 or nj < 0 or ni >= len(in10) or nj >= len(in10[ni]):
                continue
            if visited[ni][nj] > -1:
                continue
            q.append((ni, nj))
            visited[ni][nj] = visited[i][j] + 1
    elif c == 'J':
        for di, dj in ((-1, 0), (0, -1)):
            ni, nj = i + di, j + dj
            if ni < 0 or nj < 0 or ni >= len(in10) or nj >= len(in10[ni]):
                continue
            if visited[ni][nj] > -1:
                continue
            q.append((ni, nj))
            visited[ni][nj] = visited[i][j] + 1
    elif c == '7':
        for di, dj in ((1, 0), (0, -1)):
            ni, nj = i + di, j + dj
            if ni < 0 or nj < 0 or ni >= len(in10) or nj >= len(in10[ni]):
                continue
            if visited[ni][nj] > -1:
                continue
            q.append((ni, nj))
            visited[ni][nj] = visited[i][j] + 1
    elif c == 'F':
        for di, dj in ((1, 0), (0, 1)):
            ni, nj = i + di, j + dj
            if ni < 0 or nj < 0 or ni >= len(in10) or nj >= len(in10[ni]):
                continue
            if visited[ni][nj] > -1:
                continue
            q.append((ni, nj))
            visited[ni][nj] = visited[i][j] + 1

ans = max(max(_) for _ in visited)
print(ans)

6838


### Part 2

So now you have to find the area contained within those pipes. They can also squeeze out between pipes which is going to be the tricky part. I can't just count starting from a non-pipe, since that thing could STILL not be part of the enclosed area. How do I even know if something counts as enclosed then?

## [Day 11](https://adventofcode.com/2023/day/11): Cosmic Expansion

In [63]:
in11 = parse(11, chars)

Initially I had thought you need Floyd-Warshall, but since you can just get the Manhattan distance between any two points once you adjust their X and Y, this is actually O(n^2), not O(n^3). So the steps to solving this are getting the empty rows and cols, then get all the points and modulate them correctly

In [64]:
def all_galaxy_dists(grid, expansion_factor: int=2) -> list[int]:
    cols = tuple(zip(*grid))
    empty_cols = [i for i, col in enumerate(cols) if '#' not in col]
    empty_rows = [i for i, row in enumerate(grid) if '#' not in row]
    galaxies = []
    dists = []
    for ri, row in enumerate(in11):
        indices = [i for i, x in enumerate(row) if x == '#']
        for x in indices:
            cols_to_add = bisect_left(empty_cols, x)
            rows_to_add = bisect_left(empty_rows, ri)
            galaxies.append((ri+rows_to_add*(expansion_factor-1), x+cols_to_add*(expansion_factor-1)))
    
    for g1, g2 in combinations(galaxies, 2):
        dists.append(taxi_distance(g1, g2))
    return dists

In [65]:
print(sum(all_galaxy_dists(in11, 2)))

10033566


### Part 2: Increased expansion where each empty row/col is now 100,000 away. What is the total distances now?

This actually works very well with my previous answer, I just need to rewrite the previous function to take in an "expansion factor" for how much to count empty rows/cols

In [66]:
print(sum(all_galaxy_dists(in11, 1_000_000)))

560822911938
